In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

# Load the data
url = 'https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv'
data = pd.read_csv(url)

#Preprocessing data
Amount = normalize([data['Amount']])[0]
data['Amount'] = Amount
data = data.iloc[:, 1:]
data.head()

# Split the data into features and target
X = data.drop('Class', axis=1)
y = data['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the z-value and margin of error for each sampling technique
z = 1.96  # 95% confidence interval
m = 0.05  # margin of error

# Calculate the sample size for each sampling technique using the formula
n1 = int(np.ceil((z**2 * 0.5 * 0.5) / (m**2)))
n2 = int(np.ceil((z**2 * 0.05 * (1-0.05)) / (m**2)))
n3 = int(np.ceil((z**2 * 0.05 * (1-0.05)) / (m**2)))
n4 = int(np.ceil((z**2 * 0.05 * (1-0.05)) / (m**2)))
n5 = int(np.ceil((z**2 * 0.05 * (1-0.05)) / (m**2)))
n = int((1.96*1.96 * 0.5*0.5)/(0.05**2))

# Define the sampling techniques and models
sampler1 = RandomUnderSampler(sampling_strategy='majority', random_state=42)
sampler2 = RandomOverSampler(sampling_strategy='minority', random_state=42)
#sampler3 = SMOTE(sampling_strategy='minority', random_state=42)
sampler3 = TomekLinks(sampling_strategy='majority')
#interval = 2
#sampler3 = data.iloc[::interval]
# convenience sampling
sampler4= pd.concat([data.head(380), data.tail(380)])
#sampler5 = NearMiss(version=3, n_neighbors=3)
sampler5 = data.sample(n=n, random_state=42)

model1 = LogisticRegression(random_state=42,max_iter=300)
#model1 = LinearRegression()
model2 = DecisionTreeClassifier(random_state=42)
model3 = RandomForestClassifier(random_state=42)
#model3 = SVC(random_state=42)
model4=KNeighborsClassifier(n_neighbors=3)
model5= GaussianNB()

# Define a dictionary to hold the sampling techniques and models
samplers = {
    'Sampling1': sampler1,
    'Sampling2': sampler2,
    'Sampling3': sampler3,
    'Sampling4': sampler4,
    'Sampling5': sampler5,
}
models = {
    'M1': model1,
    'M2': model2,
    'M3': model3,
    'M4': model4,
    'M5': model5,
}

# Evaluate each model on each sampling technique
results = {}
for sampler_name, sampler in samplers.items():
    if sampler_name == 'Sampling1':
        n = n1
    elif sampler_name == 'Sampling2':
        n = n2
    elif sampler_name == 'Sampling3':
        n = n3
    elif sampler_name == 'Sampling4':
        n = n4
    else:
        n = n5

    # Undersample or oversample the training data
    undersampler = RandomUnderSampler()
    X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)
    oversampler = RandomOverSampler()
    X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)
   # X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    
    # Limit the resampled data to the sample size
    if len(X_resampled) > n:
        X_resampled = X_resampled[:n]
        y_resampled = y_resampled[:n]
    
    for model_name, model in models.items():
        # Train the model on the resampled data
        model.fit(X_resampled, y_resampled)
        
        # Make predictions on the test data
        y_pred = model.predict(X_test)
        
        # Calculate the accuracy score
        accuracy = accuracy_score(y_test, y_pred)
        
        # Add the accuracy score to the results dictionary
        if model_name in results:
            results[model_name][sampler_name] = accuracy
        else:
            results[model_name] = {sampler_name: accuracy}
            
# Print the results
print('Results:')
print('        Sampling1   Sampling2   Sampling3   Sampling4   Sampling5')
for model_name, model_results in results.items():
    print(model_name, end='')
    for sampler_name in samplers.keys():
        if sampler_name in model_results:
            print(f'    {model_results[sampler_name]:.4f}   ', end='')
        else:
            print('              ', end='')
    print() 

Results:
        Sampling1   Sampling2   Sampling3   Sampling4   Sampling5
M1    0.9935       0.9935       0.9935       0.9935       0.9935   
M2    0.9484       0.9871       0.9871       0.9871       0.9871   
M3    0.9935       0.9935       0.9935       0.9935       0.9935   
M4    0.9871       0.9935       0.9935       0.9935       0.9935   
M5    0.9806       0.9935       0.9935       0.9935       0.9935   
